## What are [Jupyter Notebooks](https://jupyter.org/)?

The central part of Jupyter notebooks are cells. There are two important types of cells:

+ Markdown cells with formatted text using [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet), such as this cell.
+ Code cells for program code to be executed in the [kernel](https://jupyter.readthedocs.io/en/latest/projects/kernels.html).


To write and run your code in a cell enter in the empty field below

```R
my_string <- "Hello Jupyter"
print(my_string)
```

To executed make sure that your cursor is in the cell and press `Shift-Enter`.

For a further introduction to Jupyter we refer the user to the documentation on [Jupyter Notebooks](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html)  the [JupyterLab Interface](https://jupyterlab.readthedocs.io/en/latest/user/interface.html).

**Copy of** https://www.codepile.net/pile/1WLoQQ65 by [Ranjit K. Singh](https://www.gesis.org/institut/mitarbeiterverzeichnis/person/Ranjit.Singh)

In [ ]:
library(patchwork)
library(tidyverse)

Creating the normal distribution curve

In [ ]:
normal_dist_curve_tab <- tibble(
  z = seq(-3.5,3.5,0.01),
  norm_curve = dnorm(z),
  cumulative_F = pnorm(z)
)


# Creating sine wave values from -1 to 1
# This creates the easing effect; faster animation in the middle, slow down at the edges
# Each value is thus one frame

sin_tab <- tibble(
  x = seq(-1,1,0.02),
  sin_test = sinpi(x)
)

sin_sequence <- sin_tab$sin_test

In [ ]:
for(i in c(1: length(sin_sequence))){
  # for loop once for each frame
  
  sin_val = sin_sequence[[i]]
  
  # The thresholds (vertical lines on normal curve) are defined here.
  # They are shifted left and right by the sine wave value between -1 and 1 SD
  
  thresholds <- c(-1.5, -0.5, 0.5, 1.5)+ sin_sequence[i]
  
  
  
  # Here, the normal distribution is "cut" into the thresholds
  bar_tab <- normal_dist_curve_tab %>% 
    mutate(
      bar_z_cat = cut(z, c(-Inf, thresholds, Inf)),
      bar_numeric_cat = as.integer(bar_z_cat)
    )%>% 
    group_by(bar_numeric_cat) %>% 
    summarise(density = sum(norm_curve)/100,
              mean_pos = mean(z))
  
  # Lower plot showing curve and thresholds
  
  curve_plot <-   normal_dist_curve_tab %>% 
    ggplot(aes(z, norm_curve))+
    geom_area(fill="#FBEBC3")+
    geom_point(size=2, aes(color=z))+
    geom_vline(xintercept = thresholds, size = 2, color = "#ffffff")+
    geom_vline(xintercept = thresholds, size = 1, color = "#497593")+
    geom_text(aes(x = mean_pos, y = -0.06, label = bar_numeric_cat), data = bar_tab,
              size = 8, color = "#497593")+
    scale_x_continuous(breaks = seq(-5,5, by = 0.5))+
    scale_y_continuous(limits = c(-0.1,0.45))+
    ggtitle("\n\n")+
    theme_void()+
    theme(legend.position = "none")+
    scale_colour_gradient2(low = "#189B9C", mid = "#ACA972", high = "#E6AF02")
  
  # Upper plot, showing bar chart
  
  bar_plot <- bar_tab %>% 
    ggplot(aes(bar_numeric_cat, density))+
    geom_bar(stat="identity", width = 0.5,
             fill = "#497593")+
    geom_text(aes(y = -0.05, label = bar_numeric_cat), 
              size = 8, color = "#497593")+
    scale_x_continuous(limits = c(0,6))+
    scale_y_continuous(limits = c(-0.08,0.4))+
    #ggtitle("Observed Scores")+
    theme_void()
  
  
  # Combining plots via patchwork package
  bar_plot / curve_plot
  
  # Saving to image file
  ggsave(file=paste0("frame_",i,".png"), width=10, height=5, dpi=100, type = "cairo")
  
}